In [82]:
from urllib.request import urlretrieve
import requests
from bs4 import BeautifulSoup
url = "https://search.musinsa.com/ranking/best?u_cat_cd=002"

In [83]:
html_page = requests.get(url).content
soup = BeautifulSoup(html_page, 'html.parser')
soup

<!DOCTYPE html>

<html lang="ko">
<head>
<title>상품 랭킹 | 무신사 스토어</title>
<!-- Google Tag Manager -->
<script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({'gtm.start':
            new Date().getTime(),event:'gtm.js'});var f=d.getElementsByTagName(s)[0],
        j=d.createElement(s),dl=l!='dataLayer'?'&l='+l:'';j.async=true;j.src=
        'https://www.googletagmanager.com/gtm.js?id='+i+dl;f.parentNode.insertBefore(j,f);
    })(window,document,'script','dataLayer','GTM-TXDSFSF');</script>
<!-- End Google Tag Manager -->
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="무신사 랭킹은 상품 매출, 판매 수량, 상품 조회 수, 작성 후기 수를 반영한 공식에 의해 선정됩니다.
무신사 스토어는 광고 목적으로 랭킹을 절대 임의 조작하지 않으므로 믿고 구매하셔도 됩니다." name="description"/>
<script type="text/javascript">
            var THIS_PAGE_GF = 'A';
        </script>
<link href="https://image.msscdn.net/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
<meta content="648837775133366" id="fbAddId" property="fb:app_id"/>
<meta 

In [84]:
products = soup.select("#goodsRankList > li") # goodsRankList id로 자식 li 모두 선택
products

[<li class="li_box" data-goods-no="2081554">
 <p class="n-label label-default txt_num_rank">
             1위
             
         </p>
 <div class="box-icon-right">
 <span class="icon-box-limited icon-reverse">한정 판매</span>
 </div>
 <div class="icon_group">
 <ul>
 <li class="icon_man sight_out" title="남성">남성</li>
 </ul>
 </div>
 <div class="li_inner">
 <div class="list_img">
 <a class="img-block" href="https://store.musinsa.com/app/goods/2081554?loc=goods_rank">
 <img alt="노스페이스(THE NORTH FACE) NJ1DM62A 남성 1996 에코 눕시 자켓" class="lazyload lazy" data-original="https://image.msscdn.net/images/goods_img/20210823/2081554/2081554_2_125.jpg" src="https://image.msscdn.net/0.gif"/>
 </a>
 </div>
 <div class="article_info">
 <p class="item_title">
 <a href="https://www.musinsa.com/brands/thenorthface">노스페이스</a>
 </p>
 <p class="list_info">
 <a href="https://store.musinsa.com/app/goods/2081554?loc=goods_rank" title="NJ1DM62A 남성 1996 에코 눕시 자켓">
                         
                         
 

In [85]:
len(products)

90

In [86]:
product_idx = []
product_brand_list = []
product_name_list = []
product_price_list = []
product_star_count_list = []

for product in products:

    # 브랜드
    product_brand = product.select_one("p.item_title").text
    
    
    # 상품명
    product_name = product.select_one("p.list_info").text.strip()

    # 가격
    product_price = product.select_one("p.price").text.split()
    product_price = float(product_price[-1].replace(",", "").replace("원", ""))

    # 별점 등록 수. 별점이 있는 상품이 있고, 없는 상품이 있다.
    start_count_elem = product.select_one("p.point > span.count")
    
    # 별점이 있으면 가져오고
    if start_count_elem:
        star_count = int(start_count_elem.text.replace(",", ""))
    else:
        # 별점이 없으면 0으로 설정
        star_count = 0

    # 상품 id
    product_id = product['data-goods-no']
        
    product_idx.append(product_id)
    product_brand_list.append(product_brand.strip())
    product_name_list.append(product_name.strip())
    product_price_list.append(product_price)
    product_star_count_list.append(star_count)

In [87]:
import pandas as pd
datas = {
    "ID" : product_idx,
    "brand": product_brand_list,
    "name": product_name_list,
    "price": product_price_list,
    "rate_count": product_star_count_list
}

df = pd.DataFrame(datas)
df.head()

,ID,brand,name,price,rate_count
0,2081554,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1932
1,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
2,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
3,2105483,노스페이스,NC1DM73A 고 프리 하이브리드 다운 코트 EX,269100.0,285
4,2059078,케이투,씬에어(THINAIR) ECO LITE 다운 자켓 미드나잇 블랙,295000.0,872


In [88]:
import pymongo

In [89]:
client = pymongo.MongoClient("mongodb://52.79.237.132:27017/")
client

MongoClient(host=['52.79.237.132:27017'], document_class=dict, tz_aware=False, connect=True)

In [90]:
musinsa = client.crawling.musinsa
musinsa

Collection(Database(MongoClient(host=['52.79.237.132:27017'], document_class=dict, tz_aware=False, connect=True), 'crawling'), 'musinsa')

In [91]:
insert_items = df.to_dict("records")
insert_items

[{'ID': '2081554',
  'brand': '노스페이스',
  'name': 'NJ1DM62A 남성 1996 에코 눕시 자켓',
  'price': 299000.0,
  'rate_count': 1932},
 {'ID': '865862',
  'brand': '무신사 스탠다드',
  'name': '캐시미어 블렌드 오버사이즈 싱글 코트 [블랙]',
  'price': 139900.0,
  'rate_count': 16135},
 {'ID': '2014490',
  'brand': '후아유',
  'name': '리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U',
  'price': 79920.0,
  'rate_count': 1642},
 {'ID': '2105483',
  'brand': '노스페이스',
  'name': 'NC1DM73A 고 프리 하이브리드 다운 코트 EX',
  'price': 269100.0,
  'rate_count': 285},
 {'ID': '2059078',
  'brand': '케이투',
  'name': '씬에어(THINAIR) ECO LITE 다운 자켓 미드나잇 블랙',
  'price': 295000.0,
  'rate_count': 872},
 {'ID': '2108965',
  'brand': '디아프바인',
  'name': 'DV.LOT 657 SHADOW B-3 Mouton Jacket -Cream- 21ver.',
  'price': 1740000.0,
  'rate_count': 2},
 {'ID': '2037171',
  'brand': '스파오',
  'name': '[허니푸퍼] 리버서블 푸퍼_SPJDB4VC14',
  'price': 69900.0,
  'rate_count': 4208},
 {'ID': '1535337',
  'brand': '코드그라피',
  'name': '[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_쿨그레이',
  'price': 124000.0,
  'rate

In [92]:
musinsa.insert_many(insert_items)

In [104]:
results = musinsa.find()
pd.DataFrame(results)

,_id,ID,brand,name,price,rate_count
0,61b04282f7e52fef86761132,2081554,노스페이스,NJ1DM62A 남성 1996 에코 눕시 자켓,299000.0,1932
1,61b04282f7e52fef86761133,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
2,61b04282f7e52fef86761134,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
3,61b04282f7e52fef86761135,2105483,노스페이스,NC1DM73A 고 프리 하이브리드 다운 코트 EX,269100.0,285
4,61b04282f7e52fef86761136,2059078,케이투,씬에어(THINAIR) ECO LITE 다운 자켓 미드나잇 블랙,295000.0,872
...,...,...,...,...,...,...
85,61b04282f7e52fef86761187,2087980,도프제이슨,12 / 17 배송\n \n ...,148000.0,109
86,61b04282f7e52fef86761188,2144229,쿠어,오버사이즈 마리오벨루치 더블 코트 (카멜),399600.0,9
87,61b04282f7e52fef86761189,1635197,무신사 스탠다드,[프리미엄]캐시미어 20 오버사이즈 싱글 코트 [블랙],209900.0,304
88,61b04282f7e52fef8676118a,2239457,노스페이스,NC1DM80K 화이트라벨 여성 닐튼 다운 코트,406600.0,2


In [117]:
avg = df["price"].mean()
avg

260661.11111111112

In [118]:
QUERY = { "price" : {"$lte" : avg}}
results = musinsa.find(QUERY)
pd.DataFrame(results)

,_id,ID,brand,name,price,rate_count
0,61b04282f7e52fef86761133,865862,무신사 스탠다드,캐시미어 블렌드 오버사이즈 싱글 코트 [블랙],139900.0,16135
1,61b04282f7e52fef86761134,2014490,후아유,리버서블 후드 웜업 [아메리카 블랙] WHJJB4T02U,79920.0,1642
2,61b04282f7e52fef86761138,2037171,스파오,[허니푸퍼] 리버서블 푸퍼_SPJDB4VC14,69900.0,4208
3,61b04282f7e52fef86761139,1535337,코드그라피,[PRAUDEN] 유틸리티 덕다운 푸파 숏패딩_쿨그레이,124000.0,4317
4,61b04282f7e52fef8676113b,1611891,라퍼지 포 우먼,울리치 트렌치 맥코트_Black,108000.0,2695
5,61b04282f7e52fef8676113c,2064478,라퍼지스토어,[이동휘 착용][리뉴얼 ver.][3M 신슐레이트]오리지널 M-1965 피쉬테일 파...,99900.0,7572
6,61b04282f7e52fef8676113e,1173366,라퍼지스토어,12 / 10 배송\n \n ...,89900.0,6092
7,61b04282f7e52fef86761141,411209,알파 인더스트리,MA-1 레귤러 Black,225000.0,7519
8,61b04282f7e52fef86761142,2057805,내셔널지오그래픽,N214UDW920 헤론 라이트 덕다운 유 넥 다운 점퍼 CARBON BLACK,149000.0,1614
9,61b04282f7e52fef86761143,2064480,라퍼지스토어,[이동휘 착용][리뉴얼 ver.][3M 신슐레이트]오리지널 M-1965 피쉬테일 파...,99900.0,1537
